# Практическое задание 7

In [2]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd

In [3]:
response = requests.get('https://lifehacker.ru/kak-vybrat-obogrevatel/')
response.text

'<!doctype html>\n<html data-n-head-ssr lang="ru-RU" data-n-head="%7B%22lang%22:%7B%22ssr%22:%22ru-RU%22%7D%7D">\n  <head >\n    <title>Какой обогреватель выбрать, чтобы точно было тепло — Лайфхакер</title><meta data-n-head="ssr" charset="utf-8"><meta data-n-head="ssr" name="viewport" content="width=device-width, initial-scale=1"><meta data-n-head="ssr" name="format-detection" content="telephone=no,date=no,address=no,email=no,url=no"><meta data-n-head="ssr" data-hid="al:ios:app_name" name="al:ios:app_name" content="Лайфхакер"><meta data-n-head="ssr" name="apple-itunes-app" content="app-id=943373933"><meta data-n-head="ssr" data-hid="al:ios:app_store_id" name="al:ios:app_store_id" content="943373933"><meta data-n-head="ssr" data-hid="og:publisher" property="og:publisher" content="//www.facebook.com/lifehacker.ru"><meta data-n-head="ssr" data-hid="og:site_name" property="og:site_name" content="Лайфхакер"><meta data-n-head="ssr" data-hid="og:type" property="og:type" content="website"><met

##    Определить формат ссылки для пагинации - 1 балл
Смотрим как устроены адреса на нужных страницах и определяем шаблон для пагинации

In [10]:
base_url = 'https://lifehacker.ru/topics/technology/' # базовая часть ссылки, позже в цикле будем добавлять к ней пагинацию
response = requests.get('https://lifehacker.ru/topics/technology/?page=2') # получаем контент первой страиниц
soup = BeautifulSoup(response.text, 'lxml') # инициализируем объект bs4 и задаем парсер lxml

##    В разметке страницы найти уникальные классы/идентификаторы блока с названием и содержанием материала- 3 балла
У нас материалам соответствует класс '''lh-small-article-card__link'''

In [13]:
soup.find_all('a', class_='lh-small-article-card__link') # ищем все a-элементы с классом lh-small-article-card__link

[<a aria-label="Casio выпустила новую модель часов G-Shock DW5600, вдохновлённую эстетикой Apple 80-х" class="lh-small-article-card__link" data-jest="link" href="/predstavleny-casio-g-shock-dw5600/" title="Casio выпустила новую модель часов G-Shock DW5600, вдохновлённую эстетикой Apple 80-х"></a>,
 <a aria-label="Названы самые популярные смартфоны в разных странах мира. Угадаете лидеров?" class="lh-small-article-card__link" data-jest="link" href="/top-smartfonov-v-mire/" title="Названы самые популярные смартфоны в разных странах мира. Угадаете лидеров?"></a>,
 <a aria-label="Какой обогреватель выбрать, чтобы точно было тепло" class="lh-small-article-card__link" data-jest="link" href="/kak-vybrat-obogrevatel/" title="Какой обогреватель выбрать, чтобы точно было тепло"></a>,
 <a aria-label="Microsoft представила ПК размером с Mac mini, который работает на облачной версии Windows" class="lh-small-article-card__link" data-jest="link" href="/anons-windows-365-link/" title="Microsoft предста

##    Получить содержимое десяти страниц списка материалов, выделить из него ссылки на каждый материал - 2 балла

##    Перебрать все полученные ссылки и получить html-код каждого материала - 1 балл

In [16]:
raw_items = soup.find_all('a', class_='lh-small-article-card__link')  # ищем все a-элементы с классом tm-article-comments-counter-link__link
links = [item.get('href') for item in raw_items] # получаем у ссылок только href-атрибут, убираем подстроку '/comments'
links

['/predstavleny-casio-g-shock-dw5600/',
 '/top-smartfonov-v-mire/',
 '/kak-vybrat-obogrevatel/',
 '/anons-windows-365-link/',
 '/rezervnaya-kopiya-iphone/',
 '/devaisy-ot-dreame-rasprodazha-chernaya-pyatnitza/',
 '/anons-asus-rog-phone-9/',
 '/otlichnye-umnye-vesy/',
 '/corsair-k65-plus-m75-mac/',
 '/novyi-umnyi-svet-ot-yandeksa/',
 '/luchshie-prilozheniya-i-igry-dlya-android-2024/',
 '/anons-oppo-reno-12f-harry-potter-edition/',
 '/apple-mozhet-vypustit-sobstvennyi-televizor/',
 '/kak-uznat-imei-telefona/',
 '/anons-xiaomi-ultra-slim-power-bank-5000mah/',
 '/mini-prilozheniya-2-0-telegram/',
 '/freelance-instruments/',
 '/skryt-pochtu-na-android/',
 '/casio-predstavila-chasy-kolco-crw-001-1jr-s-sekundomerom-i-migayushhim-budilnikom/',
 '/smartfony-s-originalnym-dizainom/',
 '/chatgpt-dlya-windows/',
 '/obzor-sravnenie-xiaomi-14t-i-14t-pro/',
 '/ii-agent-openai/',
 '/kakoi-vertikalnyi-pylesos-luchshe/',
 '/arc-android/',
 '/avtonomnost-xiaomi-15-pro/',
 '/razvertyvanie-hyperos-2/',
 '/

In [29]:
parsed_urls = []
for page_num in range(1, 10):
    url = f'{base_url}/?page={page_num}' # подставляем нужный номер страницы для пагинации, дальше парсим каждую страницу аналогично коду выше
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    raw_items = soup.find_all('a', class_='lh-small-article-card__link')
    parsed_urls += [item.get('href') for item in raw_items]

##    Распарсить полученный текст разметки с помощью BeautifulSoup, вытащив по каждой ссылке заголовок и содержание материала - 2 баллa
Текст

In [30]:
base_url = 'https://lifehacker.ru' # все русскоязычные материалы имеют ссылки вида https://habr.com/ru/post/718996/

response = requests.get('https://lifehacker.ru/anons-windows-365-link/')
soup = BeautifulSoup(response.text, 'lxml')
soup.find('div', class_='single-article__post-content single-article__content-container').text

'Microsoft анонсировала Windows 365 Link — напоминающий Mac mini миниатюрный ПК, предназначенный для работы с облачным сервисом Windows 365. Устройство представляет собой лёгкий в использовании клиент, позволяющий подключаться к виртуальным компьютерам, размещённым в облаке.Изображение: MicrosoftWindows 365 Link — это бесшумное устройство, которое подключается к локальным мониторам и периферии. Его основная задача — обеспечить максимально простой доступ к ПК, находящимся в облаке. По словам главы Windows и Surface Павана Давулури, компания стремилась сделать устройство максимально простым для пользователя, минимизировав количество шагов от включения до подключения к виртуальной машине.Устройство работает на облегчённой версии Windows, которая запускается за считаные секунды. Локально поддерживается только базовая операционная система, настроенная для быстрого и безопасного подключения к Windows 365. Система является «суперзащищённой» версией Windows: её код изолирован, а все приложения

Заголовок

In [31]:
soup.find('h1', class_='article-card__title').text

'Microsoft представила ПК размером с Mac mini, который работает на облачной версии Windows'

Достаем текст и заголовок каждой статьи

In [32]:
result = []
for url in tqdm(parsed_urls):
    article = {}
    article_url = f'{base_url}{url}' # подставляем часть ссылки, ведущую материал
    response = requests.get(article_url)
    soup = BeautifulSoup(response.text, 'lxml')

    article['title'] = soup.find('h1', class_='article-card__title').text # Получаем текст из span в первом h1 с классом tm-title_h1
    article['text'] = soup.find('div', class_='single-article__post-content single-article__content-container').text # Получаем весь текст из div с классом tm-article-body
   
    result.append(article)


  1%|▎                                         | 1/125 [00:00<00:38,  3.24it/s]


ConnectionError: HTTPSConnectionPool(host='lifehacker.ruhttps', port=443): Max retries exceeded with url: //lifehacker.ru/special/vesna-v-koree/ (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000000000A766198>: Failed to establish a new connection: [Errno 11004] getaddrinfo failed'))

##    Создать датайфрейм с полученными данными - 1 балл

In [33]:
pd.set_option('display.max_colwidth', 400)


data = pd.DataFrame(result)
data.head()

,text,title
0,"Портал «Грамота.ру» назвал «вайб» словом 2024 года. В шорт-листе было 11 слов, значимость и влияние которых оценивали 400 экспертов-филологов из разных регионов России.Сначала кандидаты шли с минимальным отрывом в один-два голоса, короткое время лидировал «скуф», но уже через сутки голосования «вайб» вырвался вперёд и больше позиций не сдавал. Константин Деревянко Руководителя портала «Грамота...",«Скуф» позади: «вайб» — слово года по версии «Грамота.ру»
